# scRFE Tutorial


Here we present an example of how to use scRFE. We analyze the Limb Muscle Facs data from the Tabula-Muris-Senis dataset that is available on Figshare. We split the data by age.

More features were selected than ideal in this model, because we used a very small number of estimators and a low CV score, for time's sake. This results are not accurate though, and we recommend running the code with 1000 estimators and CV>=5 with an EC2 instance.

### Imports 

In [7]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

### Read in anndata file 

In [8]:
adata = read_h5ad('/Users/madelinepark/Downloads/Limb_Muscle_facs.h5ad')
tiss = adata

In [24]:
list(set(tiss.obs['age']))

['3m', '24m']

### Run scRFE

we decreased n_estimators and cv so that the code will run faster, but you should increase both before using

In [22]:
tiss.obs['age_type_of_interest'] = 'rest'
results_age_cv = pd.DataFrame()

for c in list(set(tiss.obs['age'])): 
    print(c)
    clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration  
    age_of_interest = c
    
    tiss.obs.loc[tiss.obs[tiss.obs['age'] == age_of_interest].index,'age_type_of_interest'] = age_of_interest
    
    feat_labels = tiss.var_names 
    X = tiss.X
    y = tiss.obs['age_type_of_interest']
    
    print('training...')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0) 
    
    clf.fit(X_train, y_train)
    selector.fit(X_train, y_train)
    feature_selected = feat_labels[selector.support_] 
    
    print('result writing')

    column_headings = []
    column_headings.append(c)
    column_headings.append(c + '_gini')
    
    resaux = pd.DataFrame(columns=column_headings)
    resaux[c] = feature_selected
    resaux[c + '_gini'] = (selector.estimator_.feature_importances_)
    
    print(feature_selected)
    print (selector.estimator_.feature_importances_)
    
    results_age_cv = pd.concat([results_age_cv,resaux],axis=1)
    
    tiss.obs['age_type_of_interest'] = 'rest'
    
results_age_cv

3m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['0610007P08Rik', '0610007P14Rik', '0610007P22Rik', '0610008F07Rik',
       '0610009B14Rik', '0610009B22Rik', '0610009D07Rik', '0610009L18Rik',
       '0610009O20Rik', '0610010B08Rik',
       ...
       'Zfp101', 'Zfp187', 'Zfp189', 'Zfp207', 'Zfp212', 'Zfp281', 'Zfp874a',
       'Zmynd8', 'Zranb1', 'zsGreen_transgene'],
      dtype='object', name='index', length=4583)
[0. 0. 0. ... 0. 0. 0.]
24m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['0610007P08Rik', '0610007P14Rik', '0610007P22Rik', '0610008F07Rik',
       '0610009B14Rik', '0610009B22Rik', '0610009D07Rik', '0610009L18Rik',
       '0610009O20Rik', '0610010B08Rik',
       ...
       'Zfp101', 'Zfp187', 'Zfp189', 'Zfp207', 'Zfp212', 'Zfp281', 'Zfp874a',
       'Zmynd8', 'Zranb1', 'zsGreen_transgene'],
      dtype='object', name='index', length=4583)
[0. 0. 0. ... 0. 0. 0.]


3m   3m_gini                24m  24m_gini
0         0610007P08Rik  0.000000      0610007P08Rik  0.000000
1         0610007P14Rik  0.000000      0610007P14Rik  0.000000
2         0610007P22Rik  0.000000      0610007P22Rik  0.000000
3         0610008F07Rik  0.000000      0610008F07Rik  0.000000
4         0610009B14Rik  0.000000      0610009B14Rik  0.000000
5         0610009B22Rik  0.000166      0610009B22Rik  0.000166
6         0610009D07Rik  0.000304      0610009D07Rik  0.000304
7         0610009L18Rik  0.000000      0610009L18Rik  0.000000
8         0610009O20Rik  0.000000      0610009O20Rik  0.000000
9         0610010B08Rik  0.000000      0610010B08Rik  0.000000
10        0610010F05Rik  0.000000      0610010F05Rik  0.000000
11        0610010K14Rik  0.000519      0610010K14Rik  0.000519
12        0610010O12Rik  0.000000      0610010O12Rik  0.000000
13        0610011F06Rik  0.000000      0610011F06Rik  0.000000
14        0610011L14Rik  0.000399      0610011L14Rik  0.000399
15        0610012G03Rik  0.000000      0610012G03Rik  0.000000
16        0610012H03Rik  0.000000      0610012H03Rik  0.000000
17        0610030E20Rik  0.000000      0610030E20Rik  0.000000
18        0610031J06Rik  0.000000      0610031J06Rik  0.000000
19        0610031O16Rik  0.000000      0610031O16Rik  0.000000
20        0610037L13Rik  0.000000      0610037L13Rik  0.000000
21        0610037P05Rik  0.000164      0610037P05Rik  0.000164
22        0610038B21Rik  0.000000      0610038B21Rik  0.000000
23        0610038L08Rik  0.000000      0610038L08Rik  0.000000
24        0610039K10Rik  0.000000      0610039K10Rik  0.000000
25        0610040B10Rik  0.000000      0610040B10Rik  0.000000
26        0610040F04Rik  0.000000      0610040F04Rik  0.000000
27        0610040J01Rik  0.000000      0610040J01Rik  0.000000
28        0610043K17Rik  0.000000      0610043K17Rik  0.000000
29        0910001L09Rik  0.000000      0910001L09Rik  0.000000
...                 ...       ...                ...       ...
4553              Ywhae  0.000000              Ywhae  0.000000
4554              Ywhaz  0.000348              Ywhaz  0.000348
4555              Zbed6  0.000000              Zbed6  0.000000
4556             Zbtb20  0.001983             Zbtb20  0.001983
4557              Zbtb4  0.000000              Zbtb4  0.000000
4558             Zbtb48  0.000000             Zbtb48  0.000000
4559             Zc3h10  0.000000             Zc3h10  0.000000
4560            Zc3h11a  0.000000            Zc3h11a  0.000000
4561             Zc3h14  0.000000             Zc3h14  0.000000
4562             Zc3h15  0.000000             Zc3h15  0.000000
4563            Zc3hav1  0.000420            Zc3hav1  0.000420
4564             Zcchc2  0.000673             Zcchc2  0.000673
4565              Zcrb1  0.000179              Zcrb1  0.000179
4566              Zdbf2  0.000000              Zdbf2  0.000000
4567            Zdhhc20  0.000000            Zdhhc20  0.000000
4568             Zdhhc3  0.000000             Zdhhc3  0.000000
4569               Zeb1  0.000765               Zeb1  0.000765
4570               Zeb2  0.000000               Zeb2  0.000000
4571             Zfand5  0.001561             Zfand5  0.001561
4572             Zfand6  0.000000             Zfand6  0.000000
4573             Zfp101  0.000000             Zfp101  0.000000
4574             Zfp187  0.000000             Zfp187  0.000000
4575             Zfp189  0.000000             Zfp189  0.000000
4576             Zfp207  0.000331             Zfp207  0.000331
4577             Zfp212  0.000000             Zfp212  0.000000
4578             Zfp281  0.000000             Zfp281  0.000000
4579            Zfp874a  0.000000            Zfp874a  0.000000
4580             Zmynd8  0.000000             Zmynd8  0.000000
4581             Zranb1  0.000000             Zranb1  0.000000
4582  zsGreen_transgene  0.000000  zsGreen_transgene  0.000000

[4583 rows x 4 columns]